In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf

In [6]:
dir_data = './data'

words_path = os.path.join(dir_data, 'words.csv')
print('Path of read in data: %s' % (words_path))

fonts_pos_path = os.path.join(dir_data, 'fonts_pos.csv')
print('Path of read in data: %s' % (fonts_pos_path))

Path of read in data: ./data/words.csv
Path of read in data: ./data/fonts_pos.csv


In [7]:
'''
pic_names = []
with open('chinese_fonts.txt', 'r') as f: 
    data = f.readlines()
    for str in data:
        str = str.replace('\n','')
        pic_names.append(str)

df = pd.DataFrame({'Fonts':pic_names})
df.to_csv('Fonts.csv')
df
print(len(df))
print(df.head())

print(df.iloc[5])
'''

df_words = pd.read_csv(words_path)
print(len(df_words))
print(df_words.head())
print(df_words.iloc[[3], [1]])

pd_fonts = pd.read_csv(fonts_pos_path)
print(len(pd_fonts))
print(pd_fonts.head())

y_one_hot = np.zeros([len(df_words), len(df_words)])
# print(y_one_hot)
print(len(y_one_hot))
for i in range(len(df_words)):
    y_one_hot[i, i] = 1
print(y_one_hot)

fonts = pd_fonts['Fonts'].values
pics = df_words['Pics'].values
words = df_words['Words'].values

8877
   Unnamed: 0           Pics Words
0           0  uni4E00_一.png     一
1           1  uni4E0A_上.png     上
2           2  uni4E0B_下.png     下
3           3  uni4E0C_丌.png     丌
4           4  uni4E0D_不.png     不
            Pics
3  uni4E0C_丌.png
22
   Unnamed: 0               Fonts
0           0              AaBuYu
1           1             AaGHZKS
2           2  AaHaohaohaoxiangni
3           3       AaMangxiaoguo
4           4        AaMengMengti
8877
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [15]:
'''
x_trains = []
for i in range(len(fonts)):
    x_train = []
    dir_font = './data/training_data/positive_data/'+fonts[i]+'/'
    for j in range(len(words)):
        im = Image.open(dir_font + pics[j])
        im = np.array(im) #convert to an array
        im2 = (im/255).astype(float) #normalise input
        x_train.append(im)
    x_trains.append(x_train)    
x_trains = np.array(x_trains).reshape(len(fonts), -1, 64*64)
np.save('data/x_trains.npy', x_trains)
'''

y_trains = []
for i in range(len(fonts)):
    y_train = []
    dir_font = './data/training_data/positive_data/'+fonts[i]+'/'
    for j in range(len(words)):
        y_train.append(y_one_hot[j])
    x_trains.append(x_train)
    y_trains.append(y_train)
y_trains = np.array(y_trains)
# np.save('data/y_trains.npy', y_trains)

x_trains = np.load('data/x_trains.npy')
# y_trains = np.load('data/y_trains.npy')
print(x_trains.shape)
print(y_trains.shape)
    

(22, 8877, 4096)
(22, 8877, 8877)


In [ ]:
x = tf.placeholder(tf.float32, [None, 64*64])
x_img = tf.reshape(x, [-1, 64, 64, 1])
y = tf.placeholder(tf.float32, [None, len(df_words)])
keep_prob = tf.placeholder(tf.float32)

conv1 = tf.layers.conv2d(inputs=x_img, filters=128, kernel_size=5, strides=1, padding='SAME', activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv1, pool_size=2, strides=2)
dropout1 = tf.nn.dropout(pool1, keep_prob=keep_prob)

conv2 = tf.layers.conv2d(conv1, filters=256, kernel_size=5, strides=1, padding='SAME', activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2, pool_size=2, strides=2)
dropout2 = tf.nn.dropout(pool2, keep_prob=keep_prob)

flat = tf.reshape(dropout2, [-1, 16*16*256])
hypothesis = tf.layers.dense(flat, len(df_words))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=y))
# hypothesis = tf.nn.softmax(tf.matmul(flat, w_fc) + b_fc)
# cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(hypothesis), axis=1))

train =  tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

writer = tf.summary.FileWriter("./logs")
writer.add_graph(sess.graph)
cost_sum = tf.summary.scalar("cost", cost)
summary = tf.summary.merge_all()

def accuracy():
    predict = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))
    # x_test = mnist.test.images
    # y_test = mnist.test.labels
    x_test = x_trains[1]
    y_test = y_trains[1]
    print("test accuracy: ", sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))

training_epochs = 2
batch_size = 100
global_step = 0
print("Learning stared. It takes sometime.")
for epoch in range(training_epochs):
    total_batch = 10
    avg_cost = 0
    for batch in range(total_batch):
        x_train = x_trains[0]
        y_train = y_trains[0]
        
        permutation = np.random.permutation(x_train.shape[0])
        x_train = x_train[permutation]
        y_train = y_train[permutation]
        
        s, c, _ = sess.run([summary, cost, train], feed_dict={x:x_train, y:y_train, keep_prob:0.7})
        avg_cost += c/total_batch
        writer.add_summary(s, global_step=global_step)
        global_step += 1
    print("epoch %2d" % epoch, "avg_cost %.9f" % avg_cost)
    accuracy()
print("Learning Finished!")
    
# training_epochs = 2
# batch_size = 100
# global_step = 0
# print("Learning stared. It takes sometime.")
# for epoch in range(training_epochs):
#     total_batch = int(mnist.train.num_examples/batch_size)
#     avg_cost = 0
#     for batch in range(total_batch):
#         x_train_batch, y_train_batch = mnist.train.next_batch(batch_size)
#         s, c, _ = sess.run([summary, cost, train], feed_dict={x:x_train_batch, y:y_train_batch, keep_prob:0.7})
#         avg_cost += c/total_batch
#         writer.add_summary(s, global_step=global_step)
#         global_step += 1
#     print("epoch %2d" % epoch, "avg_cost %.9f" % avg_cost)
#     accuracy()
# print("Learning Finished!")

sess.close()

Learning stared. It takes sometime.
